In [61]:
import numpy as np
import csv
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder


#reads in the data
data = pd.read_csv('movie_metadata.csv')

#removes the values that aren't numeric
to_drop = ['director_name', 'num_critic_for_reviews', 'actor_2_name','actor_1_name', 'movie_title', 'num_voted_users'
           , 'actor_3_name', 'plot_keywords', 'movie_imdb_link', 'num_user_for_reviews', 'content_rating']

#makes the new data set without the to_drop colums
features_list = data.columns.difference(to_drop)
movie_data = data[features_list]
#print(np.sum(movie_num.isnull()))
#print(movie_data.content_rating.unique())
pd.options.mode.chained_assignment = None 
   
print('Done')

Done


# Changing Genres into individual Columns

In [62]:
genres = []

for string in movie_data['genres']:
    genre = string.split('|')
    genres = genres+genre
    
genres_set = set(genres)
print(genres_set)

{'Game-Show', 'Family', 'History', 'Crime', 'Horror', 'Western', 'Fantasy', 'News', 'Mystery', 'Documentary', 'Sci-Fi', 'Comedy', 'Reality-TV', 'Short', 'War', 'Musical', 'Thriller', 'Biography', 'Action', 'Animation', 'Music', 'Film-Noir', 'Sport', 'Drama', 'Adventure', 'Romance'}


In [63]:
genres_dict = dict()
for genre in genres_set:
    genres_dict[genre] = []
    
for string in movie_data.genres:
    genres = string.split('|')
    for genre in genres_set:
        if genre in genres:
            genres_dict[genre] = genres_dict[genre]+[1]
        else:
            genres_dict[genre] = genres_dict[genre]+[0]

del movie_data['genres']
#print(genres_dict['Short'])

for genre in genres_set:
    series = pd.Series(genres_dict[genre])
    movie_data[genre] = series

#print(movie_data['Short'].values)    
print('Done')

Done


# Deleting NaN's

In [64]:
NA_THRESH = 4

#deletes the row when there are more or equal to the threshod number of NaN's
def remove_too_many_NaN(data, threshold):
    print("Length before deletion: {}".format(len(data)))

    remove_indices = []
    for index, nNaN in data.isnull().sum(axis=1).iteritems():
        if nNaN >= threshold:
            remove_indices.append(index)
    
    # drop movies with too many NaNs
    data = data.drop(data.index[remove_indices])
    print("Length after deletion: {}".format(len(data)))
    
    return data

#returns length
movie_data = remove_too_many_NaN(movie_data, NA_THRESH)

Length before deletion: 5043
Length after deletion: 4927


# Replacing NaNs with averages

In [65]:
def replace_NaNs(col):
    # compute average
    avg = np.sum(col) / (len(col) - np.sum(col.isnull()))
    print(avg)
    
    # replace NaNs with average
    col = col.fillna(value=avg)
    return col

pd.options.mode.chained_assignment= None

to_replace_NaNs = ['actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes', 
               'aspect_ratio', 'budget', 'cast_total_facebook_likes', 'director_facebook_likes', 'gross', 'title_year'
                   , 'duration', 'facenumber_in_poster']
for column in to_replace_NaNs:
    movie_data[column] = replace_NaNs(movie_data[column])


6615.96954933
1676.81555962
651.352522376
2.13102265372
39838985.8163
9802.01867262
689.276128508
48503328.2327
2002.44376652
108.152580252
1.36561546287


# Changing Color to Numeric

In [66]:
#replaces the color value to 1
movie_data.color = movie_data.color.replace(to_replace = 'Color', value = 1)
movie_data.color = movie_data.color.replace(to_replace = 'NaN', value = 1)

#replaces the NaN or black and white values to 0
for item in movie_data.color:
    if item != 1:
        movie_data.color = movie_data.color.replace(to_replace = item, value = 0)
        
#movie_data.color = new_color_column 
#print(movie_data['color'])

#makes sure that there is nog error where it shouldn't be
pd.options.mode.chained_assignment= None
print('Done')

Done


# Changing Country to Numeric

In [67]:
#replaces the USA values to 1
movie_data.country = movie_data.country.replace(to_replace ='USA', value = 1)
movie_data.country = movie_data.country.replace(to_replace ='NaN', value = 1)
#replaces the NaN or non USA values to 0
for item in movie_data.country:
    if item != 1:
        movie_data.country = movie_data.country.replace(to_replace = item, value = 0)
       
#print(movie_data['country'])

#makes sure that there is nog error where it shouldn't be
pd.options.mode.chained_assignment= None

print('Done')

Done


# Changing Languange to Numeric

In [68]:
#replaces the English values to 1
movie_data.language = movie_data.language.replace(to_replace = 'English', value = 1)
movie_data.language = movie_data.language.replace(to_replace = 'NaN', value = 1)

#replaces the other values to 0
for item in movie_data.language:
    if item != 1:
        movie_data.language = movie_data.language.replace(to_replace = item, value = 0)
        
#print(movie_data['language'])

#makes sure that there is nog error where it shouldn't be
pd.options.mode.chained_assignment= None
print('Done')

Done


# Chaning Content_Rating to Numeric

In [69]:
def content_to_numerical(data):
    data = data.replace(to_replace='G', value=0)
    data = data.replace(to_replace='PG', value=12)
    data = data.replace(to_replace='PG-13', value=13)
    data = data.replace(to_replace='R', value=17)
    data = data.replace(to_replace='NC-17', value=17)
    
    data = data.replace(to_replace='TV-PG', value=12)
    data = data.replace(to_replace='TV-MA', value=17)
    data = data.replace(to_replace='TV-G', value=0)
    data = data.replace(to_replace='TV-Y', value=0)
    data = data.replace(to_replace='TV-Y7', value=7)
    data = data.replace(to_replace='TV-14', value=14)
    
    data = data.replace(to_replace='Not Rated', value=0)
    data = data.replace(to_replace='Unrated', value=0)
    data = data.replace(to_replace='Approved', value=0)
    data = data.replace(to_replace='Passed', value=0)
    
    data = data.replace(to_replace='X', value=17)
    data = data.replace(to_replace='M', value=17)
    data = data.replace(to_replace='GP', value=12)
    
    return data

movie_data = content_to_numerical(movie_data)
print(movie_data)

      actor_1_facebook_likes  actor_2_facebook_likes  actor_3_facebook_likes  \
0                     1000.0                   936.0                   855.0   
1                    40000.0                  5000.0                  1000.0   
2                    11000.0                   393.0                   161.0   
3                    27000.0                 23000.0                 23000.0   
5                      640.0                   632.0                   530.0   
6                    24000.0                 11000.0                  4000.0   
7                      799.0                   553.0                   284.0   
8                    26000.0                 21000.0                 19000.0   
9                    25000.0                 11000.0                 10000.0   
10                   15000.0                  4000.0                  2000.0   
11                   18000.0                 10000.0                   903.0   
12                     451.0            

# Delete columns with too little 1's

In [70]:
for genre in genres_set:
    print(genre, ':')
    counter = 0
    for value in movie_data[genre]:
        if value == 1.0:
            counter+=1
    print(counter)
    
weghalen = ['Game-Show', 'News', 'Reality-TV', 'Short', 'Film-Noir']

for name in weghalen:
    del movie_data[name]
    
#print(movie_data)

Game-Show :
0
Family :
534
History :
204
Crime :
860
Horror :
557
Western :
97
Fantasy :
593
News :
3
Mystery :
476
Documentary :
113
Sci-Fi :
600
Comedy :
1834
Reality-TV :
0
Short :
5
War :
208
Musical :
132
Thriller :
1383
Biography :
292
Action :
1132
Animation :
234
Music :
213
Film-Noir :
6
Sport :
181
Drama :
2522
Adventure :
911
Romance :
1088


# Trainset en testset

In [71]:
ratings = movie_data['imdb_score'].values
del movie_data['imdb_score']

X = movie_data.values
X_std = StandardScaler().fit_transform(X)

number_of_samples = len(ratings)
np.random.seed(0)
random_indices = np.random.permutation(number_of_samples)
num_training_samples = int(number_of_samples*0.75)
movie_training = X_std[random_indices[:num_training_samples]]
ratings_training = ratings[random_indices[:num_training_samples]]


movie_test = X_std[random_indices[num_training_samples:]]
ratings_test = ratings[random_indices[num_training_samples:]]
ratings_training = list(ratings_cont_training)


# Leren

### neural networks

In [73]:
from sklearn.neural_network import MLPRegressor

neural = MLPRegressor(alpha = 0.00001)#activation='logistic')

neural.fit(movie_training, ratings_training)
y_neural_1 = neural.predict(movie_training)
y_neural_2 = neural.predict(movie_test)
print(ratings_test)
print(y_neural_2)
error = 0

for i in range(len(ratings_training)):
    error += (abs(y_neural_1[i]-ratings_training[i]))/ratings_training[i]
training_error = error/len(ratings_training)*100

print("Train error = "+'{}'.format(training_error)+" percent"+" in neural network algorithm")

error = 0

for i in range(len(ratings_test)):
    error += (abs(y_neural_2[i]-ratings_test[i]))/ratings_test[i]
test_error = error/len(ratings_test)*100

print("Test error = "'{}'.format(test_error)+" percent"+" in neural network algorithm")

[ 7.   8.1  6.5 ...,  6.3  6.7  4.5]
[ 6.96645246  7.80039012  6.80945349 ...,  5.99125765  6.75287037
  5.73962391]
Train error = 8.69755067890222 percent in neural network algorithm
Test error = 12.785660757311728 percent in neural network algorithm


### k-nearest neighbors

In [74]:
regressor = KNeighborsRegressor()
regressor.fit(movie_training, ratings_training)
y_predict_train = regressor.predict(movie_training)
y_predict_train = list(y_predict_train)

error = 0

for i in range(len(ratings_cont_training)):
    error += (abs(y_predict_train[i]-ratings_training[i]))/ratings_training[i]
training_error = error/len(ratings_training)*100

print("Train error = "+'{}'.format(training_error)+" percent"+" in knn algorithm")

y_predict_test = regressor.predict(movie_test)
y_predict_test = list(y_predict_test)

error = 0

for i in range(len(ratings_test)):
    error += (abs(y_predict_test[i]-ratings_test[i]))/ratings_test[i]
test_error = error/len(ratings_test)*100

print("Test error = "'{}'.format(test_error)+" percent"+" in knn algorithm")

Train error = 10.276569481664392 percent in knn algorithm
Test error = 13.105962187581573 percent in knn algorithm


In [77]:
from sklearn.svm import SVR
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_lin = SVR(kernel='linear', C=1e3)
svr_poly = SVR(kernel='poly', C=1e3, degree=2)
y_rbf = svr_rbf.fit(movie_training, ratings_training).predict(movie_training)
y_rbf = list(y_rbf)
error =0

for i in range(len(ratings_training)):
    error +=(abs(y_rbf[i]-ratings_training[i])/ratings_training[i])
    
train_error= error/len(ratings_training)*100
print ("Train error = "+'{}'.format(train_error))+"%"

y2_rbf = svr_rbf.predict(movie_test)
y2_rbf = list(y2_rbf)
error =0
for i in range(len(ratings_test)):
    error +=(abs(y2_rbf[i]-ratings_test[i])/ratings_test[i])
test_error= error/len(ratings_test)*100
print ("Test error = "+'{}'.format(test_error))+"%"

SyntaxError: invalid syntax (<ipython-input-77-16a5e2e9db6a>, line 10)